In [ ]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import h5py
import matplotlib.pyplot as plt
'''import inflect
c2h.stringify = inflect.engine()'''
from tqdm import tqdm
from collections import Counter
import csv_to_hdf5 as c2h
import obspy
import math
import creep_event_picker as cep
from obspy import UTCDateTime

In [ ]:
directory = '../../Data/DATA_tidied/HDF5/'  # Replace with your actual directory path
files = c2h.list_files_in_directory(directory)
print(files)
#del files[10]
files.sort()
#del files[1],files[25],files[25],files[25],files[25],files[25],files[25],files[25],files[29],files[37],files[42],files[45],files[45],files[53],files[53],files[54]
#print(files)

In [ ]:
#files = ['SH30.h5']

In [ ]:
frequency_dict = {
        1/60: '1S',    # 1 second
        1/6: '10S',  # 10 seconds
        1/2: '30S',  # 30 seconds
        1: '1T',  # 1 minute
        2: '2T',  # 2 minutes
        5: '5T',  # 5 minutes
        10: '10T',# 10 minutes
        15: '15T', # 15 minutes
        30: '30T',# 30 minutes
        60: '60T'    # 60 minutes
    }

In [ ]:
'''def interpolate(time,slip, freq):
    creeping_df = pd.DataFrame({'Time':time,'Tm':time,'Slip':slip})
    creeping_df.Time = creeping_df.Time.dt.round(freq) #round creep times to nearest mins (make evenly spaced)
    creeping_df.Tm = creeping_df.Tm.dt.round(freq)
    creeping_df.set_index('Time',inplace=True) #set index of the dataframe
    creeping_df.drop_duplicates(subset=['Tm'], inplace=True) 
    upsampled = creeping_df.resample(freq).ffill(1)
    return upsampled'''

def interpolate(time, slip, freq):
    creeping_df = pd.DataFrame({'Time': time, 'Slip': slip})
    creeping_df['Time'] = creeping_df['Time'].dt.round(freq)
    creeping_df.set_index('Time', inplace=True)

    creeping_df = creeping_df.groupby(creeping_df.index).mean()

    # Forward-fill gaps
    upsampled = creeping_df.resample(freq).ffill(limit=1)  # optional: limit=1 prevents long fills
    upsampled['Tm'] = upsampled.index  # if needed downstream
    return upsampled

In [ ]:
Creepmeter_dataframe_SAC = pd.DataFrame()

In [ ]:
'''try:
    Creepmeter_dataframe_SAC = pd.read_csv('../../Data/DATA_tidied/creepmeter_metadata_post_standardisation_sac_codes.csv',index_col=0)
except FileNotFoundError:
    Creepmeter_dataframe_SAC = pd.DataFrame()'''


In [ ]:
Creepmeter_dataframe_SAC

In [ ]:
for i in range(len(files)):
    abbreviation = files[i][:4]
    if abbreviation[2]=='.':
        abbreviation = abbreviation[:2]
    print(abbreviation,i)
    try:
        # Open the HDF5 file in read mode
        with h5py.File('../../Data/DATA_tidied/HDF5/{k}.h5'.format(k=abbreviation), 'r') as f:
            try:
                author = f.attrs['author']
            except KeyError:
                print('no author')
            network = f.attrs['network']
            print(network)
            latitude = f.attrs['latitude']
            longitude = f.attrs['longitude']
            depth = f.attrs['depth']
            length = f.attrs['length'] 
            obliquity = f.attrs['obliquity']
            # Loop through each key in the file
            keys = list(f.keys())
            print(keys)
            
            for key in f.keys():
                if key not in ['Temperature','Temperature_1T','Temperature_5T','Daily_measurements','Manual_measurements','Orthogonal']:
                    print("Key:", key)
                    data = f[key]  # Access the dataset
                    data_keys = list(data.keys())
                    print(data_keys)
                    slip = data[data_keys[0]][:]
                    time =  data[data_keys[1]][:]
                    decoded_time = [byte_str.decode('utf-8') for byte_str in time]
                    decoded_time = pd.to_datetime(decoded_time)

                    smpl_rate = data.attrs['sampling_rate']   
                    time_units = data.attrs['time_units']     
                    slip_units = data.attrs['slip_units']  
                    channel = data.attrs['channel'] 
                    starttime = data.attrs['starttime'] 
                    delta = data.attrs['delta']
                    freq_chosen = frequency_dict[smpl_rate]
                    upsampled = interpolate(decoded_time,slip,freq_chosen)
                    print("Interpolated Slip length:", len(upsampled['Slip']))
                    print("Interpolated Time range:", upsampled['Tm'].min(), "to", upsampled['Tm'].max())
                    plt.figure()
                    plt.plot(upsampled['Tm'], upsampled['Slip'])
                    plt.title("Before SAC Write: {k}".format(k=files[i]))
                    plt.show()


                    tr = obspy.Trace(np.array(upsampled['Slip']))
                    st = obspy.Stream(tr)
                    st[0].stats.network = network
                    st[0].stats.station = abbreviation
                    st[0].stats.location = '00'
                    st[0].stats.channel = channel
                    st[0].stats.starttime = starttime
                    st[0].stats.delta = delta
                    st[0].stats['latitude'] = latitude
                    st[0].stats['longitude'] = longitude
                    st[0].stats['length'] = length
                    st[0].stats['depth'] = depth
                    st[0].stats['obliquity'] = obliquity
                    file_code = abbreviation + '_{k}'.format(k=freq_chosen)
                    print("Trace length:", len(tr.data), "Upsampled Slip length:", len(upsampled['Slip']))
                    print(file_code)
                    print(st[0].stats.endtime)
                    tr.plot();
                    instrument = pd.DataFrame({'Network':[st[0].stats.network],'Creepmeter_abbrv':['{k}'.format(k=abbreviation)],'File_code':[file_code],
                                                'Start Time':[st[0].stats.starttime],'End Time':[st[0].stats.endtime],'Sampling rate, Hz':[st[0].stats.sampling_rate],
                                                'Sampling rate, mins':[freq_chosen],'Latitude':[latitude],'Longitude':[longitude],'Length':[length],'Depth':[depth],
                                                'Obliquity':[obliquity]})
                    
                    Creepmeter_dataframe_SAC = pd.concat([Creepmeter_dataframe_SAC,instrument],ignore_index=True)
                    
                    Creepmeter_dataframe_SAC.drop_duplicates(subset='File_code',inplace=True)
                    Creepmeter_dataframe_SAC.reset_index(inplace=True,drop=True)
                    
                    #Creepmeter_dataframe_SAC.to_csv('../../Data/DATA_tidied/creepmeter_metadata_post_standardisation_sac_codes.csv')
                    #st.write('../../Data/DATA_tidied/SAC/{k}.SAC'.format(k=file_code),format='SAC')
    except:
        print('no file found')
            

In [ ]:
creepmeter_list = pd.read_excel('../../Data/Creepmeter_list.xlsx')
creepmeter_list

In [ ]:
full_name = []
for i in range(len(Creepmeter_dataframe_SAC)):
    abbrv = Creepmeter_dataframe_SAC['Creepmeter_abbrv'].iloc[i]
    print(abbrv)
    instrument_df = creepmeter_list[creepmeter_list['Creepmeter_abbrv'] == abbrv]
    full_name.append(instrument_df['Creepmeter_full_name'].iloc[0])

print(full_name)
Creepmeter_dataframe_SAC['Creepmeter_full_name'] = full_name

In [ ]:
Creepmeter_dataframe_SAC

In [ ]:
Creepmeter_dataframe_SAC.to_csv('../../Data/DATA_tidied/creepmeter_metadata_post_standardisation_sac_codes_updated.csv')

In [ ]:
st = obspy.read('../../Data/DATA_tidied/SAC/DU30_1T.SAC')
st[0].stats

In [ ]:
data.keys

In [ ]:
st[0].plot()

In [ ]:
with h5py.File('../../Data/DATA_tidied/HDF5/SIV1.h5', 'r+') as f:
    keys = list(f.keys())
    print(keys)
    old_group_name = 'SIV1_0.16667mins'
    new_group_name = 'SIV1_10s'
    old_dataset_name1 = 'Slip_SIV1_0.16667mins'
    new_dataset_name1 = 'Slip_SIV1_10s'
    old_dataset_name2 = 'Time_SIV1_0.16667mins'
    new_dataset_name2 = 'Time_SIV1_10s'
     # Check if the old group and dataset exist
    if old_group_name in f:
        # Create a new group
        new_group = f.create_group(new_group_name)
        #new_group = f[new_group_name]

        new_group.attrs['sampling_rate'] = f[old_group_name].attrs['sampling_rate']  # Example: 100 Hz sampling rate (10 ms interval)
        new_group.attrs['time_units'] = 'minutes'     # Time units
        new_group.attrs['slip_units'] = 'millimetres'  # Dependent variable units (e.g., for slip)
        new_group.attrs['channel'] = 'slip'
        new_group.attrs['starttime'] = f[old_group_name].attrs['starttime']
        new_group.attrs['delta'] = f[old_group_name].attrs['delta']
        # Copy the dataset from the old group to the new group
        if old_dataset_name1 in f[old_group_name]:
            data = f[old_group_name][old_dataset_name1][:]
            new_group.create_dataset(new_dataset_name1, data=data)

        
        # Copy the dataset from the old group to the new group
        if old_dataset_name2 in f[old_group_name]:
            data = f[old_group_name][old_dataset_name2][:]
            new_group.create_dataset(new_dataset_name2, data=data)

    

In [ ]:
with h5py.File('../../Data/DATA_tidied/HDF5/SIV1.h5', 'r+') as f:
    keys = list(f.keys())
    print(keys)
    del f['SIV1_0.16667mins']
    keys = list(f.keys())
    print(keys)

In [ ]:
with h5py.File('../../Data/DATA_tidied/HDF5/FXC1.h5', 'r+') as f:
    keys = list(f.keys())
    print(keys)
    old_group_name = 'FXC1_1.0mins'
    new_group_name = 'FCR1_1.0mins'
    old_dataset_name1 = 'Slip_FXC1_1.0mins'
    new_dataset_name1 = 'Slip_FCR1_1.0mins'
    old_dataset_name2 = 'Time_FXC1_1.0mins'
    new_dataset_name2 = 'Time_FCR1_1.0mins'
     # Check if the old group and dataset exist
    if old_group_name in f:
        # Create a new group
        new_group = f.create_group(new_group_name)
        #new_group = f[new_group_name]

        new_group.attrs['sampling_rate'] = f[old_group_name].attrs['sampling_rate']  # Example: 100 Hz sampling rate (10 ms interval)
        new_group.attrs['time_units'] = 'minutes'     # Time units
        new_group.attrs['slip_units'] = 'millimetres'  # Dependent variable units (e.g., for slip)
        new_group.attrs['channel'] = 'slip'
        new_group.attrs['starttime'] = f[old_group_name].attrs['starttime']
        new_group.attrs['delta'] = f[old_group_name].attrs['delta']
        # Copy the dataset from the old group to the new group
        if old_dataset_name1 in f[old_group_name]:
            data = f[old_group_name][old_dataset_name1][:]
            new_group.create_dataset(new_dataset_name1, data=data)

        
        # Copy the dataset from the old group to the new group
        if old_dataset_name2 in f[old_group_name]:
            data = f[old_group_name][old_dataset_name2][:]
            new_group.create_dataset(new_dataset_name2, data=data)

In [ ]:
with h5py.File('../../Data/DATA_tidied/HDF5/FCR1.h5', 'r+') as f:
    keys = list(f.keys())
    print(keys)
    del f['FXC1_1.0mins']
    keys = list(f.keys())
    print(keys)

In [ ]:
data

In [ ]:
%matplotlib qt
with h5py.File('../../Data/DATA_tidied/HDF5/SC30.h5', 'r') as f:
    keys = list(f.keys())
    print(keys)
    plt.figure()
    for key in f.keys():
        if key not in ['Temperature','Temperature_1T','Temperature_5T','Daily_measurements','Manual_measurements','Orthogonal']:
            print("Key:", key)
            data = f[key]  # Access the dataset
            data_keys = list(data.keys())
            print(data_keys)
            slip = data[data_keys[0]][:]
            time =  data[data_keys[1]][:]
            decoded_time = [byte_str.decode('utf-8') for byte_str in time]
            decoded_time = pd.to_datetime(decoded_time)

            smpl_rate = data.attrs['sampling_rate']   
            time_units = data.attrs['time_units']     
            slip_units = data.attrs['slip_units']  
            channel = data.attrs['channel'] 
            starttime = data.attrs['starttime'] 
            delta = data.attrs['delta']
            freq_chosen = frequency_dict[smpl_rate]

            #plt.plot(decoded_time,slip)
            upsampled = interpolate(decoded_time,slip,freq_chosen)
            
            plt.plot(upsampled['Tm'],upsampled['Slip'])